###### The total data volume is too large to be processed at one time.
###### Using 2021 data as an example

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from shapely.geometry import Point, Polygon
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

## Part 1: Taxi Zones Deta

In [2]:
df = pd.read_csv("dataSource/taxi_zones.csv")

In [3]:
df.head(5)

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens
2,3,0.084341,MULTIPOLYGON (((-73.84792614099985 40.87134223...,0.000314,Allerton/Pelham Gardens,3,Bronx
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
4,5,0.092146,MULTIPOLYGON (((-74.17421738099989 40.56256808...,0.000498,Arden Heights,5,Staten Island


In [4]:
df.shape

(263, 7)

In [5]:
df.dtypes

OBJECTID        int64
Shape_Leng    float64
the_geom       object
Shape_Area    float64
zone           object
LocationID      int64
borough        object
dtype: object

- Filter relevant information outside of Manhattan.

In [6]:
filtered_df = df[df['borough'] == 'Manhattan']

- Saveing the data to a new csv file.

In [7]:
filtered_df.to_csv('programing data/taxi_zones.csv', index=False)

## Part 2: Taxi Trip Deta

### Explain the meaning of each feature

- VendorID: The identifier of the taxi provider or operator.
- tpep_pickup_datetime: The date and time of the pick-up.
- tpep_dropoff_datetime: The date and time of the drop-off.
- passenger_count: Number of passengers.
- trip_distance: Travel distance
- pickup_longitude: The longitude coordinate of the pick-up.
- pickup_latitude: The latitudinal coordinates of the pick-up.
- RatecodeID: A code that identifies the taxi rate.
- store_and_fwd_flag: Is a field in New York City taxi data that records a Boolean value indicating whether the trip was stored locally by the vehicle and subsequently forwarded to the dispatch center.
- dropoff_longitude: The longitude coordinate of the drop-off.
- dropoff_latitude: The latitudinal coordinates of the drop-off.
- payment_type: Payment method for passengers.
- fare_amount: The amount of the trip cost, excluding surcharges, tips, tolls, etc.
- extra: Additional charges, which may include parking fees, baggage fees, etc.
- mta_tax: Nyc Transit Authority (MTA) additional taxes and fees.
- tip_amount: The amount of a tip given by a passenger.
- tolls_amount: The amount of the toll.
- improvement_surcharge: An improvement surcharge, usually a fee charged under certain conditions.
- total_amount: The total cost, including the sum of the trip cost, additional fees, tips, tolls and other surcharges.
- PULocationID: The location ID of the pick-up.
- DOLocationID: The location ID of the drop-off.

### New feature

- Date: Day of data generation..
- WeekDay: day of the week
- PU_passenger: The number of passengers of pick_up.
- PUTripDistance_sum: The total distance traveled by taxis leaving the area
- PickUp_count: The count of the pick-up.
- PU_fare: The amount of the trip cost of all trips left from the area.
- PUT_fare: The total cost of the number of passengers of pick_up.
- LocationID: Manhattan Neighborhood ID.
- DO_passenger: The number of passengers of drop_off.
- DOTripDistance_count: Is the total distance traveled by the taxi to reach the area.
- Drop_off: The count of the drop-off.
- DO_fare: The amount of the trip cost of all trips reach the area..
- DOT_fare: The total cost of the number of passengers of drop_off.

### Cleaning Data

- Take 2021 as an example

In [8]:
df = pd.read_csv("dataSource/2021_Yellow_Taxi_Trip_Data.csv")

/var/folders/6g/ybj4cz9s1wd3hg435tvgpfb00000gn/T/ipykernel_67098/3297450824.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataSource/2021_Yellow_Taxi_Trip_Data.csv")


In [9]:
df.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,01/01/2021 12:30:10 AM,01/01/2021 12:36:12 AM,1.0,2.10,1.0,N,142,43,2.0,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5
1,1.0,01/01/2021 12:51:20 AM,01/01/2021 12:52:19 AM,1.0,0.20,1.0,N,238,151,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
2,1.0,01/01/2021 12:43:30 AM,01/01/2021 01:11:06 AM,1.0,14.70,1.0,N,132,165,1.0,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0
3,1.0,01/01/2021 12:15:48 AM,01/01/2021 12:31:01 AM,0.0,10.60,1.0,N,138,132,1.0,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0
4,2.0,01/01/2021 12:31:49 AM,01/01/2021 12:48:21 AM,1.0,4.94,1.0,N,68,33,1.0,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5


- Check how many rows and columns dataset has.

In [10]:
df.shape

(30904072, 18)

- Check database data types

In [11]:
df.dtypes

VendorID                 float64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count          float64
trip_distance            float64
RatecodeID               float64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type             float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

- Convert Date Format

In [12]:
df['Date'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.date

- Check if there are duplicate rows or columns

In [13]:
#remove whitespace in or around feature names
df.columns = df.columns.str.replace(' ', '')

#check to ensure whitespaces have been removed
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Date'],
      dtype='object')

In [14]:
#check for duplicate rows

#Print the number of duplicates, without the original rows that were duplicated
print('Number of duplicate (excluding first) rows in the table is: ', df.duplicated().sum())

# Use "keep=False" to mark all duplicates as true, including the original rows that were duplicated.
print('Number of duplicate rows (including first) in the table is:', df[df.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


- Check if there is a constant column

In [15]:
#Check the data of category type to see if there is a constant column
df_columns = df.columns
features_card = list(df[df_columns].columns.values)

print('{0:35}  {1}'.format("Feature", "Unique Values"))
print('{0:35}  {1}'.format("-------", "--------------- \n"))

for c in df_columns:
    print('{0:35}  {1}'.format(c, str(len(df[c].unique()))))

Feature                              Unique Values
-------                              --------------- 

VendorID                             5
tpep_pickup_datetime                 16906064
tpep_dropoff_datetime                16893686
passenger_count                      13
trip_distance                        9233
RatecodeID                           8
store_and_fwd_flag                   3
PULocationID                         263
DOLocationID                         261
payment_type                         7
fare_amount                          11834
extra                                326
mta_tax                              40
tip_amount                           5733
tolls_amount                         1844
improvement_surcharge                3
total_amount                         20759
congestion_surcharge                 14
Date                                 399


- Check %Missing column and %null column

In [16]:
# Prepare %Missing column and %null column
categorical_missing = {'Feature':[], 'Missing%':[], 'Null%':[], 'Total%':[]}
for column in df.columns:
    categorical_missing['Feature'].append(column)
    categorical_missing['Missing%'].append(100*sum(df[column]=='Missing')/df.shape[0])
    categorical_missing['Null%'].append(100*(df[column].isnull().sum())/df.shape[0])
    categorical_missing['Total%'].append((100*sum(df[column]=='Missing')/df.shape[0])+(100*(df[column].isnull().sum())/df.shape[0]))
pd.DataFrame(categorical_missing)

,Feature,Missing%,Null%,Total%
0,VendorID,0.0,2.698764,2.698764
1,tpep_pickup_datetime,0.0,0.000000,0.000000
2,tpep_dropoff_datetime,0.0,0.000000,0.000000
3,passenger_count,0.0,4.784790,4.784790
4,trip_distance,0.0,0.000000,0.000000
5,RatecodeID,0.0,4.784790,4.784790
6,store_and_fwd_flag,0.0,4.784790,4.784790
7,PULocationID,0.0,0.000000,0.000000
8,DOLocationID,0.0,0.000000,0.000000
9,payment_type,0.0,2.698764,2.698764


From the above results, we can see that ‘VendorID’, 'passenger_count', 'RatecodeID', 'store_and_fwd_flag', 'payment_type', 'congestion_surcharge' have missing values.

- 'VendorID ', 'RatecodeID', 'store_and_fwd_flag', 'payment_type', 'congestion_surcharge ', although they are not constant columns, However, their Unique Values are small and cannot be filled according to other values. Moreover, they have no influence on the busy degree determination of different regions, so they should be remove.

In [17]:
drop_column = ['VendorID', 'RatecodeID', 'store_and_fwd_flag', 'payment_type','congestion_surcharge']
df = df.drop(drop_column, axis = 1)

- 'passenger_count' needs to be reserved, using Median filling method to figure out how many passengers have the same departure and destination on average.

In [18]:
empty_passenger = df[df['passenger_count'].isnull()]
rows_drop = empty_passenger.index
df = df.drop(rows_drop)

In [19]:
df.shape

(29425377, 14)

In [20]:
empty_passenger.head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Date
1272093,01/01/2021 11:32:00 AM,01/01/2021 11:39:00 AM,NaN,1.26,91,71,27.44,0.0,0.5,2.75,0.0,0.3,30.99,2021-01-01
1272094,01/01/2021 11:01:48 AM,01/01/2021 11:01:04 AM,NaN,1.68,265,36,24.94,0.0,0.5,0.00,0.0,0.3,25.74,2021-01-01
1272095,01/01/2021 11:01:43 AM,01/01/2021 11:01:03 AM,NaN,3.67,265,37,17.12,0.0,0.5,0.00,0.0,0.3,17.92,2021-01-01
1272096,01/01/2021 11:01:00 AM,01/01/2021 11:21:00 AM,NaN,7.42,41,79,27.73,0.0,0.5,2.75,0.0,0.3,31.28,2021-01-01
1272097,01/01/2021 11:21:00 AM,01/01/2021 11:43:00 AM,NaN,3.22,22,71,19.32,0.0,0.5,2.75,0.0,0.3,22.87,2021-01-01


In [21]:
# Calculate passenger average by different PULocationID and DOLocationID
passenger = df.groupby(['PULocationID', 'DOLocationID'])['passenger_count'].mean().reset_index()

In [22]:
empty_passenger.drop(labels = ['passenger_count'], axis = 1, inplace = True)
# Use the merge method to match against 'PULocationID' and 'DOLocationID' and get the corresponding 'passenger_count'
merged_df = empty_passenger.merge(passenger, on=['PULocationID', 'DOLocationID'], how='left')

In [23]:
merged_df.head(5)

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Date,passenger_count
0,01/01/2021 11:32:00 AM,01/01/2021 11:39:00 AM,1.26,91,71,27.44,0.0,0.5,2.75,0.0,0.3,30.99,2021-01-01,1.200000
1,01/01/2021 11:01:48 AM,01/01/2021 11:01:04 AM,1.68,265,36,24.94,0.0,0.5,0.00,0.0,0.3,25.74,2021-01-01,1.000000
2,01/01/2021 11:01:43 AM,01/01/2021 11:01:03 AM,3.67,265,37,17.12,0.0,0.5,0.00,0.0,0.3,17.92,2021-01-01,1.333333
3,01/01/2021 11:01:00 AM,01/01/2021 11:21:00 AM,7.42,41,79,27.73,0.0,0.5,2.75,0.0,0.3,31.28,2021-01-01,1.341651
4,01/01/2021 11:21:00 AM,01/01/2021 11:43:00 AM,3.22,22,71,19.32,0.0,0.5,2.75,0.0,0.3,22.87,2021-01-01,1.000000


- Merge the data back into the original table

In [24]:
df = pd.concat([df, merged_df], ignore_index=True)

In [25]:
merged_df = None
passenger = None

- Check %Missing column and %null column again

In [26]:
# Prepare %Missing column and %null column
categorical_missing = {'Feature':[], 'Missing%':[], 'Null%':[], 'Total%':[]}
for column in df.columns:
    categorical_missing['Feature'].append(column)
    categorical_missing['Missing%'].append(100*sum(df[column]=='Missing')/df.shape[0])
    categorical_missing['Null%'].append(100*(df[column].isnull().sum())/df.shape[0])
    categorical_missing['Total%'].append((100*sum(df[column]=='Missing')/df.shape[0])+(100*(df[column].isnull().sum())/df.shape[0]))
pd.DataFrame(categorical_missing)

,Feature,Missing%,Null%,Total%
0,tpep_pickup_datetime,0.0,0.000000,0.000000
1,tpep_dropoff_datetime,0.0,0.000000,0.000000
2,passenger_count,0.0,0.119253,0.119253
3,trip_distance,0.0,0.000000,0.000000
4,PULocationID,0.0,0.000000,0.000000
5,DOLocationID,0.0,0.000000,0.000000
6,fare_amount,0.0,0.000000,0.000000
7,extra,0.0,0.000000,0.000000
8,mta_tax,0.0,0.000000,0.000000
9,tip_amount,0.0,0.000000,0.000000


- Due to this data can also provide other information, such as the LocationID of the car being pick up, so keep the.

### Selecting features：

- Look for rows and columns. Consider whether it makes sense to keep them or drop them.

Feature Selection Summary:
- VendorID: The identifier of the taxi provider or operator, the feature is obviously not useful for analysis. Drop the feature.
- tpep_pickup_datetime: Retention year data
- tpep_dropoff_datetime: Retention year data
- passenger_count: Number of passengers, save.
- trip_distance: Travel distance, save.
- RatecodeID: Final rate code, drop it.
- store_and_fwd_flag: Is a field in New York City taxi data that records a Boolean value indicating whether the trip was stored locally by the vehicle and subsequently forwarded to the dispatch center. The feature is obviously not useful for analysis. Drop the feature.
- payment_type: The feature is obviously not useful for analysis. Drop the feature.
- fare_amount: It is part of the total amount and is not helpful for analysis, drop.
- extra: It is part of the total amount and is not helpful for analysis, drop.
- mta_tax: It is part of the total amount and is not helpful for analysis, drop.
- tip_amount: It is part of the total amount and is not helpful for analysis, drop.
- tolls_amount: It is part of the total amount and is not helpful for analysis, drop.
- improvement_surcharge: It is part of the total amount and is not helpful for analysis, drop.
- total_amount: The total amount to be paid, save.
- PULocationID: The location ID of the pick-up, save.
- DOLocationID: The location ID of the drop-off, save.

- Drop unselected columns

In [27]:
columns_to_drop = [ "extra", "mta_tax", "tip_amount"]
df = df.drop(columns_to_drop, axis=1)

- Summarize information: According to the LocationID summary information, the count of pick up, the count of drop off, the number of passengers of pick_up, the number of passengers of drop_off, the total cost of the number of passengers of pick_up, and the total cost of the number of passengers of drop_off in different regions are counted.

- Stats based on pick_up

In [28]:
pick_df = df.groupby(['Date','PULocationID']).agg({
    'passenger_count': 'sum',
    'trip_distance': 'sum',
    'PULocationID': 'count',
    'fare_amount': 'sum',
    'total_amount': 'sum',
}).rename(columns={
    'trip_distance': 'PUTripDistance_count',
    'PULocationID': 'DropOff_count',
    'passenger_count': 'PU_passenger',
    'fare_amount': 'PU_fare',
    'total_amount': 'PUT_fare',
    'PULocationID': 'PickUp_count'
}).reset_index()

In [29]:
pick_df.head(5)

,Date,PULocationID,PU_passenger,PUTripDistance_count,PickUp_count,PU_fare,PUT_fare
0,2002-12-31,193,1.0,0.00,1,0.0,0.0
1,2003-01-01,68,1.0,5.53,1,18.5,22.3
2,2003-01-01,140,1.0,7.59,1,22.5,30.3
3,2003-01-01,236,1.0,1.34,1,7.5,11.3
4,2003-01-01,237,1.0,2.07,1,9.0,12.8


- Stats based on drop_off

In [30]:
drop_df = df.groupby(['Date','DOLocationID']).agg({
    'passenger_count': 'sum',
    'trip_distance': 'sum',
    'DOLocationID': 'count',
    'fare_amount': 'sum',
    'total_amount': 'sum',
}).rename(columns={
    'trip_distance': 'DOTripDistance_count',
    'DOLocationID': 'DropOff_count',
    'passenger_count': 'DO_passenger',
    'fare_amount': 'DO_fare',
    'total_amount': 'DOT_fare',
    'PULocationID': 'DropOff_count'
}).reset_index()

In [31]:
drop_df.head(5)

,Date,DOLocationID,DO_passenger,DOTripDistance_count,DropOff_count,DO_fare,DOT_fare
0,2002-12-31,193,1.0,0.00,1,0.0,0.0
1,2003-01-01,25,1.0,7.59,1,22.5,30.3
2,2003-01-01,75,1.0,5.53,1,18.5,22.3
3,2003-01-01,140,1.0,1.34,1,7.5,11.3
4,2003-01-01,246,1.0,2.07,1,9.0,12.8


- Merging the pick_df and drop_df tables

In [32]:
merged_df = pd.merge(left=pick_df, right=drop_df, left_on=['Date', 'PULocationID'], right_on=['Date', 'DOLocationID'], how='inner')

In [33]:
merged_df.head(5)

,Date,PULocationID,PU_passenger,PUTripDistance_count,PickUp_count,PU_fare,PUT_fare,DOLocationID,DO_passenger,DOTripDistance_count,DropOff_count,DO_fare,DOT_fare
0,2002-12-31,193,1.0,0.00,1,0.0,0.0,193,1.0,0.00,1,0.0,0.0
1,2003-01-01,140,1.0,7.59,1,22.5,30.3,140,1.0,1.34,1,7.5,11.3
2,2003-01-03,193,1.0,0.00,1,0.0,0.0,193,1.0,0.00,1,0.0,0.0
3,2003-01-05,193,1.0,0.00,1,0.0,0.0,193,1.0,0.00,1,0.0,0.0
4,2003-01-05,264,1.0,0.00,1,0.0,0.0,264,1.0,0.00,1,0.0,0.0


- Clean up retained data.

    Since we only need data within Manhattan, excluding data from non-Manhattan pick-up and drop-off locations.

In [34]:
zone = pd.read_csv('programing data/taxi_zones.csv')

In [35]:
zone.head(5)

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
1,24,0.047000,MULTIPOLYGON (((-73.95953658899997 40.79871852...,0.000061,Bloomingdale,24,Manhattan
2,12,0.036661,MULTIPOLYGON (((-74.01565756599994 40.70483308...,0.000042,Battery Park,12,Manhattan
3,13,0.050281,MULTIPOLYGON (((-74.01244109299991 40.71905767...,0.000149,Battery Park City,13,Manhattan
4,41,0.052793,MULTIPOLYGON (((-73.94773985499985 40.80959972...,0.000143,Central Harlem,41,Manhattan


In [36]:
# Extract 'PULocationID' column as location_df
location_df = merged_df['PULocationID']
# Extract 'LocationID' column as location_zone
location_zone = zone['LocationID']
# Create a boolean mask indicating whether an element in location_df exists in location_zone or not
mask = location_df.isin(location_zone)
# Filter df based on boolean mask and keep only the rows that meet the conditions
merged_df = merged_df[mask]

In [37]:
# Remove duplicate LocationIDs
merged_df.drop(labels=['PULocationID'], axis=1,inplace=True)
merged_df = merged_df.rename(columns={'DOLocationID':'LocationID'})

- Removal of non-2021 data

In [39]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df = merged_df[merged_df['Date'].dt.year == 2021]
merged_df = merged_df.reset_index(drop=True)

- Save Data

In [40]:
merged_df.to_csv('programing data/taxi_2021.csv', index=False)

- merge all Taxi Trip Data

In [ ]:
merged_df = pd.DataFrame()
df1 = pd.read_csv('programing data/taxi_2017.csv')
merged_df = pd.concat([merged_df, df1], ignore_index=True)
df1 = pd.read_csv('programing data/taxi_2018.csv')
merged_df = pd.concat([merged_df, df1], ignore_index=True)
df1 = pd.read_csv('programing data/taxi_2019.csv')
merged_df = pd.concat([merged_df, df1], ignore_index=True)
df1 = pd.read_csv('programing data/taxi_2020.csv')
merged_df = pd.concat([merged_df, df1], ignore_index=True)
df1 = pd.read_csv('programing data/taxi_2021.csv')
merged_df = pd.concat([merged_df, df1], ignore_index=True)

- Save Data

In [ ]:
merged_df.to_csv('programing data/2017-2021.csv', index=False)